In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-12-10 14:52:25.729570: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 14:52:25.759727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 14:52:25.759751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 14:52:25.760331: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 14:52:25.765487: I tensorflow/core/platform/cpu_feature_guar

In [2]:
X_data = np.load("../data/data_64_30.npy")
X_data = np.transpose(X_data, (0, 2, 1))
print(X_data.shape)

(13100, 30, 64)


In [3]:
data = pd.read_csv(
    "../data/LJSpeech-1.1/metadata.csv",
    sep="|",
    header=None,
    names=["ID", "Text1", "Text2"],
)
texts = data["Text1"].to_list()
ID = data["ID"].to_list()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
# num_classes = len(tokenizer.word_index) + 1  # Add 1 for the padding token
sequences = tokenizer.texts_to_sequences(texts)
Y_data = pad_sequences(sequences, padding="post", maxlen=30)

In [4]:
class ConvolutionalLayer(tf.keras.layers.Layer):  # Change the base class to tf.keras.layers.Layer
    def __init__(self, input_shape, filters=32, kernel_size=3, **kwargs):
        super(ConvolutionalLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

        # Extract the number of filters from the input shape
        if isinstance(input_shape, tuple):
            self.filters = input_shape[-1]

        self.conv1 = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding="same")
        self.batch_norm1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()

        self.conv2 = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding="same")
        self.batch_norm2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()

    def call(self, inputs, training=None, mask=None):
        conv1_out = self.relu1(self.batch_norm1(self.conv1(inputs), training=training))
        conv2_out = self.relu2(self.batch_norm2(self.conv2(conv1_out), training=training))
        # print("CNN output shape is ",conv2_out.shape)
        return conv2_out


In [5]:

class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn_layer = ConvolutionalLayer(input_shape=(30, 64))  # Adjust input shape

    def call(self, inputs, training=None):
        cnn_output = self.cnn_layer(inputs, training=training)  # Explicitly call the 'call' method
        return cnn_output

In [6]:
def scaled_dot_product(q, k, v, mask):
    d_k = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_qk = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(d_k)

    if mask is not None:
        scaled_qk += mask

    attention_weights = tf.nn.softmax(scaled_qk)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [7]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = tf.keras.layers.Dense(3 * d_model, use_bias=False)
        self.linear_layer = tf.keras.layers.Dense(d_model, activation='relu')

    def split_heads(self, x, batch_size):
        if len(x.shape) == 2:
            x = tf.expand_dims(tf.expand_dims(x, axis=0), axis=1)
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, x, mask):
        batch_size, _, _ = x.shape

        qkv = self.qkv_layer(x)
        q, k, v = tf.split(qkv, 3, axis=-1)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        values, attention = scaled_dot_product(q, k, v, mask)

        values = tf.transpose(values, perm=[0, 2, 1, 3])
        values = tf.reshape(values, (batch_size, -1, self.num_heads * self.head_dim))
        out = self.linear_layer(values)
        return out


In [8]:
class PositionwiseFeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = tf.keras.layers.Dense(hidden)
        self.linear2 = tf.keras.layers.Dense(d_model)
        self.relu = tf.keras.layers.ReLU()
        self.dropout = tf.keras.layers.Dropout(rate=drop_prob)

    
    def call(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [9]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, hidden, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.cnn_layer = CNN()
        self.multihead_attention = MultiHeadAttention(d_model, num_heads)
        self.feedforward = PositionwiseFeedForward(d_model, hidden, drop_prob=dropout_rate)
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)

    def call(self, x, training=None, mask=None):
        x = self.cnn_layer(x)
        x_att = self.multihead_attention(x, mask)
        x = x + x_att
        x = self.dropout(x)
        x = self.feedforward(x)
        return x

In [10]:
class Encoder(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, hidden, dropout_rate=0.1):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.d_model = d_model
        self.num_heads = num_heads
        self.hidden = hidden
        self.dropout_rate = dropout_rate
        self.flatten = layers.Flatten()
        # self.output_layer = layers.Dense(30)
        self.output_layer = layers.Dense(30, activation='softmax')

        # Define layers
        self.encoder_layers = [EncoderLayer(d_model, num_heads, hidden, dropout_rate) for _ in range(num_layers)]

    def call(self, x, training=None, mask=None):
        for i in range(self.num_layers):
            x = self.encoder_layers[i](x, training=training, mask=mask)
        x = self.flatten(x)
        x = self.output_layer(x)
        return x

In [11]:
encoder = Encoder(num_layers=11, d_model=64, num_heads=8, hidden=128, dropout_rate=0.1)

2023-12-10 14:52:27.111019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 14:52:27.135263: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-10 14:52:27.135435: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
# encoder.compile(optimizer='adam',
#                      loss='mse',
#                      metrics=['mae'])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.25)
encoder.compile(optimizer=optimizer, loss="mae",metrics=['mae'])
# encoder.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
history = encoder.fit(X_data, Y_data, epochs=10,batch_size=50, validation_data=(X_data, Y_data))

Epoch 1/10


2023-12-10 14:52:35.807159: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-10 14:52:35.874968: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-10 14:52:36.047596: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-10 14:52:36.827811: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f658e3ec240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 14:52:36.827842: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-12-10 14:52:36.831682: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702200156.897586   19021 device_compiler.

262/262 [==============================] - 25s 34ms/step - loss: 626.9050 - mae: 626.9050 - val_loss: 626.9164 - val_mae: 626.9164
Epoch 2/10
262/262 [==============================] - 8s 31ms/step - loss: 626.9060 - mae: 626.9060 - val_loss: 626.9166 - val_mae: 626.9166
Epoch 3/10
262/262 [==============================] - 8s 31ms/step - loss: 626.9059 - mae: 626.9059 - val_loss: 626.8892 - val_mae: 626.8892
Epoch 4/10
262/262 [==============================] - 8s 31ms/step - loss: 626.8890 - mae: 626.8890 - val_loss: 626.8892 - val_mae: 626.8892
Epoch 5/10
262/262 [==============================] - 8s 31ms/step - loss: 626.8891 - mae: 626.8891 - val_loss: 626.8892 - val_mae: 626.8892
Epoch 6/10
262/262 [==============================] - 9s 33ms/step - loss: 626.8890 - mae: 626.8890 - val_loss: 626.8892 - val_mae: 626.8892
Epoch 7/10
262/262 [==============================] - 9s 35ms/step - loss: 626.8893 - mae: 626.8893 - val_loss: 626.8892 - val_mae: 626.8892
Epoch 8/10
262/262 [===